This notebook pretreats the following data for use in analysis:
* survey data collected using the infoflora application.
* survey data collected without the infoflora application
* The invasives species list, last updated in 2016, available [here](https://www.infoflora.ch/fr/neophytes/neophytes.html)
* The redlist of endangered species, updated in 2019, avaialble __.
* The list of priority species, updated in ___, available __.
* The welten sutter lists in and around Biel/Bienne
* 20 square kilomters of 5x5 observations centered on Bienne from Infoflora available __

Unneeded columns are dropped, names are homogenized and dictionary keys/codes are created for the different classifications.
* removing extraneous data (mainly columns, some rows)
* linking some other useful information to the data (species conservation status, etc)

In [2]:
# import packages

# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
# from matplotlib import colors
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

# os and file types
import os
import sys
import datetime as dt
import json
import csv

In [4]:
# invasive preprocess

# main work is to combine the columns containing the geographic distribution, the threats posed, and the habitats for the invasive species.
# note that no reference number is provided by infoflora with this data set, unlike the others.

# load the invasive species data from infoflora
invasive = pd.read_excel("data/BL_WL_2014_f_v2020_05_18.xls", header = 2)

# drop 4 empty rows of nan values at the end of the data set
invasive= invasive.iloc[:-4]

# turn threat columns in invasive db into more useable framework. The resulting column is a list containing the threats specified in mythreats.
# note result are strings of integers, not integers!

mythreats = ["Potentiel d'expansion",'écologie, biodiversité']

mylist = []

for i in invasive.index:
    mysublist = []
    for j in mythreats:
        if not pd.isna(invasive.loc[i,j]):
            mysublist.append([j,invasive.at[i,j]])
    mylist.append(mysublist)
    for k in mylist:
        for l in k:
            if "xxx" in l[1]:
                l[1] = "3"
            if "xx" in l[1]:
                l[1] = "2"
            if "x" in l[1]:
                l[1] = "1"
invasive["threats"] = mylist

# turn location of invasives columns from db into more useable framework. The resulting column is a list containing the status of the places specified in myplaces.
# note result are strings of integers, not integers!

myplaces = ['Jura', 'Plateau','non établi en Suisse']

mylist = []

for i in invasive.index:
    mysublist = []
    for j in myplaces:
        if not pd.isna(invasive.loc[i,j]):
            mysublist.append([j,invasive.at[i,j]])
    mylist.append(mysublist)
    for k in mylist:
        for l in k:
            if "xxx" in l[1]:
                l[1] = "3"
            if "xx" in l[1]:
                l[1] = "2"
            if "x" in l[1]:
                l[1] = "1"         
invasive["places"] = mylist

# collect all of the habitat data into one column. 
#In the db the presence of a species in each habitat is provided as a checkmark in the relevant column.
# For each species in the db, this code makes a list of the column names with a checkmark, and stores this data under a new "habitat" column

myhabitats = ['1 Eaux libres', '2 Rivages et lieu humides',
          '3 Glaciers, rochers, éboulis et moraines', '4 Pelouses et prairies',
          '5 Landes, lisières et mégaphorbiaies', '6 Forêts',
          '7 Végétations pionnières des endroits perturbés',
          '8 Plantations, champs, cultures', '9 Milieux construits']
mylist = []
for i in invasive.index:
    mysublist = []
    for j in myhabitats:
        if not pd.isna(invasive.loc[i,j]):
            mysublist.append(j)
    mylist.append(mysublist)
invasive["habitat"] = mylist

# keep select columns and write result to csv
keep_invasives = ['Latein','Black List / Watch List 2014','Ordonnonce sur la dissémination des organismes (ODE)', 'threats','places', 'habitat']
inv = invasive[keep_invasives].copy()
inv.to_csv("output/invasive_preproc.csv")

In [5]:
# redlist preprocess
redlist = pd.read_excel("data/CH-RLreg_Tracheophyta_2019.xlsx", engine = 'openpyxl',header = 1)

#turn the redlist criteria for each region into a function of the first letter which denotes the general reason behind its classification.
#at this stage, no need for the rest of the details
mycriteria = ['crit_CH','crit_JU','crit_MP']
mylist = []

for i in redlist.index:
    for j in mycriteria:
        if not pd.isna(redlist.loc[i,j]):
            split = redlist.at[i,j].split(";")
            split[0] = split[0][0]
            if len(split) > 1:
                if (split[1][0] == " ")|(split[1][0] == "\t"):
                    split[1] = split[1][1:]
                split[1] = split[1][0]
            redlist.at[i,j] = split

# drop columns not relevant, write to new csv file
keep_redlist = ['ID_ISFS','FAMILY','GENUS','Scientific name','CH','crit_CH', 'JU', 'crit_JU', 'MP', 'crit_MP']
red = redlist[keep_redlist].copy()
inv.to_csv("output/redlist_preproc.csv")

In [3]:
# import the data, comes in batches of 4000 observations

first = pd.read_csv("data/obs_export_2022-04-17_23h22.csv",encoding = "utf-16",sep = "\t")
second = pd.read_csv("data/obs_export_2022-04-17_23h23.csv",encoding = "utf-16",sep = "\t")

#concatonate the different datasets of observations

all_data = pd.concat([first,second],ignore_index = True)

# check data / column names

#for i in all_data.columns:
#    print(F"column {i} ++ value {all_data.at[0,i]}")
#all_data.columns

# test for NAN values

#a = all_data.altitude_min.isna()
#for i in a:
#    if i:
#        print('an NA')
#    else:
#        pass

In [6]:
# keys for interpreting redlist data
redlist_key_places = {'JU':'Jura','MP':'Middle Plateau'}
redlist_key_status = {'EN':'Endangered', 'VU': 'Vulnerable','RE':'Extinct','CR': 'Critically Edangered',
                     'NT': 'Near Threatened','LC':'Least Concern','DD':'Data deficient',
                      'NA':'Not Applicable','NE': 'Not Evaluated'}
redlist_key_criteria = {'A': 'decrease in population size','B':'Habitat fragmentation',
                        'C':'initial small population, decrease','D':'very small habitat/population size'}

# keys to interpret invasive data
key_invasives_status = {'BL':'Black List', 'WL': 'Watch List', 'ODE': 'Prohibited'}
key_invasives_threat = {'1':'low impact','2':'medium impact','3':'high  impact'}
key_invasives_places = {'1':'low frequency','2':'medium frequency','3':'high frequency'}

# key to interpret infoflora survey data

# key to interpret priority data
priority_canton_key = {'JU':'Jura','BE':'Bern'}
priority_laws_key = {'Espèce cible forestière':'','Espèce agricole OEA','Espèce endémique','Espèce Émeraude':'Bern convention','Espèce protégée':'Protected under art. 20'}

In [7]:
# priority species 

priority = pd.read_excel('data/ch_priority_species.xlsx',header = 9)

In [8]:
priority = priority[priority["Organismengruppe"] == "Gefässpflanzen"]
a = priority.columns
print(a)

Index([              'Unnamed: 0',     'Taxon ID InfoSpecies',
               'Organismengruppe',              'Org. Kürzel',
                       'Taxon ID',               'Taxon Name',
             'Nützliche Synonyme',           'Deutscher Name',
             'Französischer Name',       'Italienischer Name',
       ...
                             'UR',                       'VD',
                             'VS',                       'ZG',
                             'ZH',              'Waldzielart',
       'Landwirtschaftl. UZL-Art',           'Endemische Art',
                   'Smaragd- Art',       'Geschützte Art NHV'],
      dtype='object', length=109)


In [9]:
myhabitats = list(range(1,10))
mylist = []

for i in priority.index:
    mysublist = []
    for j in myhabitats:
        if priority.loc[i,j] == 'x':
            mysublist.append(j)
    mylist.append(mysublist)
priority["habitat"] = mylist
priority.habitat

1046       [3]
1047       [4]
1048    [4, 5]
1049       [3]
1050       [5]
         ...  
2310       [3]
2311       [3]
2312       [7]
2313       [4]
2314       [1]
Name: habitat, Length: 842, dtype: object

In [10]:
mylaws = ['Waldzielart','Landwirtschaftl. UZL-Art','Endemische Art','Smaragd- Art','Geschützte Art NHV']

mylist = []

for i in priority.index:
    mysublist = []
    for j in mylaws:
        if not pd.isna(priority.loc[i,j]):
            mysublist.append(j)
    mylist.append(mysublist)
priority["protection"] = mylist

In [11]:
keep_priority = ['Taxon ID','Taxon ID InfoSpecies','Taxon Name','Habitatkombination','Jura','Mittelland','habitat','Kollin','Montan','JU','BE','Priorität','Verantwortung','protection']
pri = priority[keep_priority].copy()
inv.to_csv("output/priority_preproc.csv")

In [12]:
redlist.crit_CH

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
2910    [C, D]
2911       [D]
2912       [C]
2913    [C, B]
2914       [A]
Name: crit_CH, Length: 2915, dtype: object

Next, the irrelevant columns are removed. InfoFlora provides documentation on the column names available [here](https://www.infoflora.ch/fr/assets/content/documents/Demande_donnees/legende_fr.pdf). The following is the list of columns downloaded: 
       
       'obs_id', 'releve_id', 'project.id', 'project.copyright',
       'project.project_name', 'releve_type', 'date', 'date_precision',
       'date_expert', 'v_accepted_taxon_id', 'taxon.taxon_id',
       'taxon.taxon_name', 'taxon.no_isfs', 'taxon_orig', 'taxon_expert',
       'determinavit_cf', 'determinavit', 'introduced', 'introduced_expert',
       'municipality.id', 'municipality.name', 'municipality_expert',
       'locality_descript', 'srid', 'x', 'y', 'geometry', 'xy_type',
       'xy_precision', 'geo_expert', 'altitude_min', 'altitude_max',
       'altitude_expert', 'specimen_type', 'presence', 'count_unit',
       'abundance_code', 'cover', 'habitat.typo_ch_id', 'habitat.name',
       'phenology', 'vitality', 'rem', 'v_validation_status', 'v_observers',
       'v_co_canton', 'documents', 'supplements.obs_type',
       'supplements.project_obs_id', 'supplements.abundance',
       'supplements.pop_length', 'supplements.pop_width',
       'supplements.plant_height', 'supplements.depth_min',
       'supplements.depth_max', 'supplements.sexe',
       'supplements.releve_stratum', 'supplements.cover_abs',
       'supplements.cat_aggregation', 'supplements.substrate',
       'last_modified_when',
The following is the list of columns that were kept:
* obs_id = kept as unique identifier
* md = transformed date 
* x = x coordinate gis system (long WS84)
* y = y coordinate gis system (lat WS84)
* taxon_id = unique species identifier for infoflora reference
* taxon_no.ifsa = another unique species identifier for infoflora reference
* taxon_orig = original species name selected
* municipality.name = name of municipality sample taken
* phenology = stage of growth
* locality_descript = name of survey site
* specimen type = if there is a photo
* v_co_canton = name of canton sample taken
* xy_precision = accuracy of x,y coordinates

Note that for many of these a dictionary has been made to key numerical values with string values (e.g. value of "20" in "cover" column corresponds to "5%<x<10%")


In [13]:
# the columns to keep out of the entire set of data.
columns_keep = ['obs_id', 'date','determinavit_cf', 'taxon.taxon_id', 'taxon_orig','municipality.id','municipality.name', 'x', 'y', 'phenology','cover','altitude_min','locality_descript','specimen_type', 'v_co_canton','xy_precision']

# make a new df out of those columns
ad = all_data[columns_keep].copy()

Initial data work

In [14]:
# transform pd object in date column into dt object

ad["md"] = pd.to_datetime(ad['date'], errors='coerce', format='%Y-%m-%d')
ad["month"] = ad["md"].dt.month
ad["year"] = ad["md"].dt.year